<a href="https://colab.research.google.com/github/adityaRajPratapSingh/Transformers_Basics/blob/main/GPT_2_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OVERVIEW

In order to generate creative texts i am going to fine GP2 model using a few books from the project gutenberg dataset. Due to the lack of compute resources at the moment i am going to start with 5 books of the same author(Jane Austen) to match the writing style.


the books i am going to be downloading are:


1.   pride and prejudice-1342
2.   emma-158
3.   sense and sensibility-161
4.   persuasion-105
5.   northanger abbey-121



In [1]:
!git clone https://github.com/kiasar/gutenberg_cleaner.git

import nltk
import sys
sys.path.append('./gutenberg_cleaner')

Cloning into 'gutenberg_cleaner'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 113 (delta 8), reused 0 (delta 0), pack-reused 98 (from 1)
Receiving objects: 100% (113/113), 29.61 KiB | 14.81 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
# this block is for dowloading the 5 books from the project's website and combining then to make a dataset
from gutenberg_cleaner import super_cleaner, simple_cleaner
import requests
import os

def download_gutenberg(book_ids:list[int])->None:
  base_url = 'https://www.gutenberg.org/files/{}/{}'

  for book_id in book_ids:
    url = base_url.format(book_id, f"{book_id}-0.txt")
    response = requests.get(url)
    if response.status_code == 200:
      cleaned = simple_cleaner(response.text)
      with open(f'book_{book_id}.txt', 'w', encoding='utf-8') as file:
        file.write(cleaned)
      print(f"book_{book_id} has been downloaded")
    else:
      print(f"book_{book_id} could not be downloaded")

  combined_dataset=""
  for book_id in book_ids:
    with open(f'book_{book_id}.txt', 'r', encoding='utf-8') as file:
      combined_dataset += file.read() + '\n'

  with open('combined_dataset.txt', 'w', encoding = 'utf-8') as file:
    file.write(combined_dataset)

download_gutenberg([1342, 158, 161, 105, 121, 141])

book_1342 has been downloaded
book_158 has been downloaded
book_161 has been downloaded
book_105 has been downloaded
book_121 has been downloaded
book_141 has been downloaded


In [3]:
!pip install -U transformers
!pip install datasets

import transformers
from nltk.tokenize import sent_tokenize
import torch
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 953.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from torch.utils.data import random_split

with open('combined_dataset.txt', 'r', encoding = 'utf-8') as file:
  data = file.read()

nltk.download('punkt')
sentences = sent_tokenize(data)
processed_sentences = [sentence.lower() for sentence in sentences]

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(processed_sentences, return_tensors="pt", truncation=True, padding=True)

model = GPT2LMHeadModel.from_pretrained("gpt2")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/results",
    per_device_train_batch_size=32,
    num_train_epochs=10,
    save_strategy="no",
    logging_dir="./logs",
    logging_steps=100,
    report_to='none',
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=inputs['input_ids'],
    data_collator=data_collator
)

trainer.train()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
100,3.858500
200,3.547200
300,3.442500
400,3.391900
500,3.360200
600,3.345500
700,3.311300
800,3.289700
900,3.189800
1000,3.159100


TrainOutput(global_step=8210, training_loss=2.886226101130719, metrics={'train_runtime': 1781.6929, 'train_samples_per_second': 147.433, 'train_steps_per_second': 4.608, 'total_flos': 5.643718046208e+16, 'train_loss': 2.886226101130719, 'epoch': 10.0})

In [9]:
model.save_pretrained("/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/gpt2_tuned")
tokenizer.save_pretrained("/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/tokenizer")

('/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/tokenizer/vocab.json',
 '/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/tokenizer/merges.txt',
 '/content/drive/MyDrive/My Colab Notebooks/Transformers_Basics/tokenizer/added_tokens.json')

In [11]:
def generate_text(prompt, model, tokenizer, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        inputs['input_ids'],
        max_length=max_length,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        attention_mask=inputs['attention_mask'],
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7,
        do_sample=True
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

prompt = "Once upon a time in a distant land, there lived a"
print(generate_text(prompt, model, tokenizer))


Once upon a time in a distant land, there lived a
young lady, whom she had been waiting for, in the course of the first
days of their acquaintance; and, with a new constitution, an object of
a much curiosity, and a very happy wish to be acquainted with her.
here it was again, on the very first morning of her arrival, when
they met again on their journey.”

“you are very kind, sir thomas;


In [15]:
from math import exp
from torch.nn.functional import cross_entropy

def calculate_perplexity(text, model, tokenizer):
    tokens = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(**tokens, labels=tokens['input_ids'])
        loss = outputs.loss
        perplexity = exp(loss.item())
    return perplexity

sample_text = "Once upon a time in a distant land, there lived a"
print("Perplexity:", calculate_perplexity(sample_text, model, tokenizer))

Perplexity: 59.28415963616727
